# Überblick über Basis-Tools: `scikit-learn`

`scikit-learn` ist das Standard-Tool für Machine Learning in Python. Fast alle *klassischen ML-Verfahren* werden durch `scikit-learn` unterstützt.

`scikit-learn` verfügt (im Gegensatz zu vielen anderen Open-Source-Paketen) über eine fantatische Dokumentation, die du dir unter https://scikit-learn.org/stable/ unbedingt anschauen solltest.

Sehr praktisch ist auch, dass `scikit-learn` bereits Datensets mitbringt, mit denen man die Funktionalität ausprobieren kann.

## Supervised Learning

Als ganz einfaches Beispiel nutzen wir hier ein Datenset, das aus 20 Newsgroups besteht. Es ist zwar schon ziemlich alt, aber die Sprache hat sich nicht sehr geändert. Das Datenset lässt sich ganz leicht laden:

In [ ]:
!pip install sklearn

In [ ]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train')

Die Daten findest du in dem Schlüssel `data` des zurückgelieferten `dict`.

In [ ]:
len(newsgroups_train['data'])

Und jetzt kannst du fast schon mit dem Machine Learning beginnen. Die Daten müssen nur noch *vektorisiert* werden. Was dabei genau passiert, schauen wir uns später noch an:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(newsgroups_train.data)

Sobald du die Vektoren zur Verfügung hast, kannst du einen Klassifikator trainieren, hier eine sog. *Support Vector Machine*. Der Klassifikator soll lernen, aufgrund eines Newsgroups-Posting die korrekte Kategorie (in diesem Fall die Newsgroup, in der gepostet wurde) zu erraten. Ein solches Problem heißt *Klassifikationsproblem*.

In [ ]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss='hinge', max_iter=1000, tol=1e-3, random_state=42)
clf.fit(vectors, newsgroups_train.target)

Das ging **schnell**. Immerhin waren es über 11.000 Texte, aus denen das Modell die Kategoriezuordnung gelernt hat.

Anhand einer unabhängigen Datenmenge kannst die Ergebnisse verfizieren:

In [ ]:
from sklearn import metrics
newsgroups_test = fetch_20newsgroups(subset='test')
vectors_test = vectorizer.transform(newsgroups_test.data)
pred = clf.predict(vectors_test)
metrics.f1_score(newsgroups_test.target, pred, average='macro')

Auch das geh sehr schnell und mit über 84% *F-Score* ziemlich genau. Wenn du dir anschauen möchtest, wo Fehler passiert sind, hilft die sog. *Confusion Matrix*, die in der Diagonale die richtigen Ergebnisse zeigt:

In [ ]:
import pandas as pd
pd.DataFrame(metrics.confusion_matrix(newsgroups_test.target, pred), 
             columns=newsgroups_train.target_names, index=newsgroups_train.target_names)

Die Verwechslungen von `alt.atheism` mit `talk.religion.misc` und `soc.religion.christian` kann man in Nicht-Diagonal-Elementen der Confusion Matrix gut erkennen. Die könnten einem Menschen auch passieren, genau wie `comp.os.ms-windows.misc` mit `comp.sys.ibm.pc.hardware` und `comp.windows.x`

## Unsupervised Learning

`scikit-learn` kann nebem *überwachtem Lernen* auch viel Funktionen zum *unüberwachten Lernen* vorweisen. Unüberwachtes Lernen kannst du dazu nutzen, die organische Struktur eines Textkorpus zu explorieren:

In [ ]:
from sklearn.decomposition import NMF
nmf = NMF(n_components=5)
nmf.fit(vectors)

In [ ]:
features = vectorizer.get_feature_names()
topics = []
for topic, word_vector in enumerate(nmf.components_):
    total = word_vector.sum()
    largest = word_vector.argsort()[::-1] # invert sort order
    
    topics.append([f" {features[largest[i]]}" for i in range(5)])
pd.DataFrame(topics, columns=[f"Wort {i}" for i in range(5)], index=[f"Topic {i}" for i in range(5)])

Hier siehst du, dass wir noch mehr Arbeit in die Vorbereitung der Daten stecken müssen. Die wichtigsten Worte der Topics sind fast alle ziemlich bedeutungslos (sog. *Stoppworte*). Lösungen dafür schauen wir uns im Laufe des Kurses an.

## `scikit-learn` kann noch viel mehr

Du hast bisher nur an der Oberfläche von `scikit-learn` gekratzt. `scikit-learn` bietet noch viel mehr Algorithmen,
Methoden und Konzepte, die wir uns in späteren Lektionen genau ansehen werden.